In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile

zip_path = "/content/drive/MyDrive/dataset.zip"  # Path to your ZIP file in Google Drive
extract_to = "/content/dataset"  # Folder where you want to extract

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(" Extraction complete!")

 Extraction complete!


In [1]:
import os, cv2, numpy as np
from tqdm import tqdm
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img

IMG_SIZE = 160
BATCH_SIZE = 512

model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
cat_path = '/content/dataset/dataset/cats'
dog_path = '/content/dataset/dataset/dogs'

def process_batch(folder, label):
    features, labels = [], []
    batch = []

    for idx, file in enumerate(tqdm(os.listdir(folder))):
        try:
            img_path = os.path.join(folder, file)
            img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
            img = img_to_array(img)
            batch.append(img)

            # Process batch
            if len(batch) == BATCH_SIZE or idx == len(os.listdir(folder)) - 1:
                batch_arr = preprocess_input(np.array(batch))
                preds = model.predict(batch_arr, verbose=0)
                features.extend(preds)
                labels.extend([label]*len(preds))
                batch = []  # Clear batch
        except:
            continue

    return features, labels

cat_features, cat_labels = process_batch(cat_path, 0)
dog_features, dog_labels = process_batch(dog_path, 1)

X = np.array(cat_features + dog_features)
y = np.array(cat_labels + dog_labels)


/tmp/ipython-input-3448218581.py:10: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


100%|██████████| 10008/10008 [01:25<00:00, 117.41it/s]


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import pickle

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

# Save
with open("svm_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("✅ Accuracy:", model.score(X_test, y_test))


✅ Accuracy: 0.9600798403193613
